In [2]:
# Параметры сбора постов
DAYS_BACK = 3                        # За сколько дней брать посты (1-60)
POSTS_PER_CHANNEL = 1                 # Сколько постов брать с каждого канала
POST_LIMIT_PER_REQUEST = 15           # Сколько постов запрашивать у Telegram (макс)
REQUEST_DELAY = 1                     # Задержка между запросами (секунды)
CHANNELS_PER_FILE = 1  # Сколько каналов брать из каждого CSV файла

# ============= ПАРАМЕТРЫ ФИЛЬТРАЦИИ =============
MIN_POST_LENGTH = 80                  # Минимальная длина поста (символов)
MAX_LINKS_PER_POST = 1                # Максимальное количество ссылок в посте
MAX_SPECIAL_CHARS = 10                # Максимальное количество спецсимволов

# ============= ПАРАМЕТРЫ СОЗДАНИЯ DATASET =============
TARGET_PAIRS = 20                   # Целевое количество пар в golden dataset
MATCHING_RATIO = 0.5                  # Доля подходящих пар (0.5 = 50/50)
MAX_ATTEMPTS_MULTIPLIER = 5           # Макс попыток = TARGET_PAIRS * этот множитель

In [3]:
import asyncio
import pandas as pd
import json
import os
import glob
import random
from datetime import datetime, timedelta
from telethon import TelegramClient
from telethon.tl.types import Message
import re

# Настройка путей для Windows
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, "data")
channels_folder = os.path.join(data_dir, "channels")
goods_folder = os.path.join(data_dir, "goods")
mapping_file = os.path.join(data_dir, "category_mapping.json")
golden_folder = os.path.join(data_dir, "golden")

# Создаем папки если их нет
for folder in [channels_folder, goods_folder, golden_folder]:
    os.makedirs(folder, exist_ok=True)

print(f"Текущая директория: {current_dir}")
print(f"Папка с каналами: {channels_folder}")
print(f"Папка с товарами: {goods_folder}")
print(f"Папка для golden dataset: {golden_folder}")
print(f"Файл с маппингом категорий: {mapping_file}")

Текущая директория: d:\Projects\thesis
Папка с каналами: d:\Projects\thesis\data\channels
Папка с товарами: d:\Projects\thesis\data\goods
Папка для golden dataset: d:\Projects\thesis\data\golden
Файл с маппингом категорий: d:\Projects\thesis\data\category_mapping.json


In [18]:
# Загружаем товары из Excel
goods_file = os.path.join(goods_folder, "goods_data.xlsx")
print(f"Загружаю товары из файла: {goods_file}")

try:
    goods_df = pd.read_excel(goods_file, engine='openpyxl')
    print(f"✓ Товары загружены. Размер: {goods_df.shape[0]} строк, {goods_df.shape[1]} столбцов")
    
    # Конвертируем в список словарей
    products = goods_df.to_dict('records')
    print('products:', products[:10])
    print(f"✓ Конвертировано товаров: {len(products)}")
    
except Exception as e:
    print(f"✗ Ошибка загрузки товаров: {e}")
    products = []

# Загружаем маппинг категорий
print(f"\nЗагружаю маппинг категорий из файла: {mapping_file}")

try:
    with open(mapping_file, 'r', encoding='utf-8') as f:
        category_mapping = json.load(f)
    print(f"✓ Маппинг загружен. Категорий: {len(category_mapping)}")
    
    # Выводим примеры маппинга
    print("\nПримеры маппинга категорий:")
    for i, (product_cat, channel_cats) in enumerate(list(category_mapping.items())[:5], 1):
        print(f"{i}. {product_cat} -> {channel_cats}")
        
except Exception as e:
    print(f"✗ Ошибка загрузки маппинга: {e}")
    category_mapping = {}

Загружаю товары из файла: d:\Projects\thesis\data\goods\goods_data.xlsx
✓ Товары загружены. Размер: 1000 строк, 9 столбцов
products: [{'imt_id': 10016198, 'nm_id': 13372527, 'imt_name': 'Футболка', 'subj_name': 'Футболки', 'subj_root_name': 'Одежда', 'nm_colors_names': 'оранжевый', 'vendor_code': 'S208оранжевый', 'description': 'Одежда ESPORT отличается лекгостью тканиЮ удобной ноской и приятная телу. Швы аккуратные,  яркие цвета, упругая и втоже время тянеться до нужной посадки.', 'brand_name': 'Esport'}, {'imt_id': 10012660, 'nm_id': 13366707, 'imt_name': 'Чехол на iPhone XS MAX TELEPRO Nano Silicone', 'subj_name': 'Чехлы для телефонов', 'subj_root_name': 'Смартфоны и аксессуары', 'nm_colors_names': 'черный', 'vendor_code': 'SC0208/IPXSMAXчерный', 'description': 'Яркий и стильный чехол TELEPRO Nano Silicone выполнен из качественного, приятного на ощупь силикона с микрофиброй внутри. Легко надевается и надежно защищает Ваш смартфон.', 'brand_name': 'TELEPRO'}, {'imt_id': 10012455, 'nm

In [10]:
def load_channels_from_csv(channels_folder, channels_per_file=CHANNELS_PER_FILE):
    """
    Загружает каналы из всех CSV файлов в папке.
    Возвращает список словарей: {'channel_id': ..., 'category': ..., 'handle': ...}
    """
    print(f"\nЗагружаю каналы из папки: {channels_folder}")
    
    channels = []
    csv_files = glob.glob(os.path.join(channels_folder, "*.csv"))
    print(f"Найдено CSV файлов: {len(csv_files)}")
    
    for file in csv_files:
        # Извлекаем категорию из имени файла
        filename = os.path.basename(file)
        
        # Убираем префиксы и расширение
        category = re.sub(r'^RU_|^ru_|_channels$|\.csv$', '', filename, flags=re.IGNORECASE)
        category = category.replace('_', ' ').strip()
        
        try:
            # Пробуем разные кодировки
            for encoding in ['utf-8', 'utf-8-sig', 'windows-1251']:
                try:
                    df = pd.read_csv(file, encoding=encoding)
                    break
                except:
                    continue
            
            # Ищем столбец с ID каналов
            channel_col = None
            for col in df.columns:
                if 'channel' in str(col).lower() or 'id' in str(col).lower():
                    channel_col = col
                    break
            
            if channel_col is None and len(df.columns) > 0:
                channel_col = df.columns[0]  # берем первый столбец
            
            if channel_col is not None:
                # Берем по несколько каналов из каждого файла
                channel_ids = df[channel_col].dropna().astype(str).str.strip().tolist()
                selected = channel_ids[:channels_per_file] if len(channel_ids) > channels_per_file else channel_ids
                
                for channel_id in selected:
                    # Находим handle для этого channel_id
                    handle = ''
                    
                    # Ищем столбец 'handle' в DataFrame
                    if 'handle' in df.columns:
                        # Находим строку с этим channel_id
                        matching_rows = df[df[channel_col].astype(str).str.strip() == channel_id]
                        if not matching_rows.empty:
                            handle_value = matching_rows.iloc[0]['handle']
                            if pd.notna(handle_value):
                                handle = str(handle_value)
                    
                    channels.append({
                        'channel_id': channel_id,
                        'category': category,
                        'source_file': filename,
                        'handle': handle
                    })
                
                print(f"  {category}: {len(selected)} каналов")
            else:
                print(f"  {category}: не найден столбец с ID каналов")
                
        except Exception as e:
            print(f"  Ошибка чтения файла {file}: {e}")
    
    print(f"\n✓ Всего загружено каналов: {len(channels)}")
    print(f"✓ Уникальных категорий каналов: {len(set(c['category'] for c in channels))}")
    
    # Статистика по handle
    channels_with_handle = sum(1 for c in channels if c['handle'])
    print(f"✓ Каналов с handle: {channels_with_handle} ({channels_with_handle/len(channels)*100:.1f}%)")
    
    return channels
             
             

# Загружаем каналы
all_channels = load_channels_from_csv(channels_folder, channels_per_file=CHANNELS_PER_FILE)


Загружаю каналы из папки: d:\Projects\thesis\data\channels
Найдено CSV файлов: 43
  apps channels: 1 каналов
  art channels: 1 каналов
  babies channels: 1 каналов
  beauty channels: 1 каналов
  blogs channels: 1 каналов
  books channels: 1 каналов
  business channels: 1 каналов
  career channels: 1 каналов
  construction channels: 1 каналов
  courses channels: 1 каналов
  crypto channels: 1 каналов
  design channels: 1 каналов
  economics channels: 1 каналов
  education channels: 1 каналов
  edutainment channels: 1 каналов
  entertainment channels: 1 каналов
  esoterics channels: 1 каналов
  food channels: 1 каналов
  gambling channels: 1 каналов
  games channels: 1 каналов
  handmade channels: 1 каналов
  health channels: 1 каналов
  instagram channels: 1 каналов
  language channels: 1 каналов
  law channels: 1 каналов
  marketing channels: 1 каналов
  medicine channels: 1 каналов
  music channels: 1 каналов
  nature channels: 1 каналов
  news channels: 1 каналов
  other channels: 1

In [ ]:
# Конфигурация Telegram API
api_id = TG_API_ID 
api_hash = TG_API_HASH
client = TelegramClient('thesis_session', api_id, api_hash)

async def get_channel_posts(channel_id: str, start_date: datetime, end_date: datetime, limit=10) -> list[Message]:
    """
    Получает посты из канала в заданном временном диапазоне.
    """
    try:
        await client.start()
        
        messages = []
        async for message in client.iter_messages(
            entity=channel_id,
            limit=POSTS_PER_CHANNEL,
            wait_time=REQUEST_DELAY
        ):
            messages.append(message)
            print(message)

        await client.disconnect()
        return messages
    except Exception as e:
        print(f"    Ошибка при получении постов из {channel_id}: {e}")
        await client.disconnect()
        return []

def filter_non_ad_posts(posts):
    """
    Фильтрует посты, оставляя только нерекламные с нормальным текстом.
    """
    filtered = []
    ad_keywords = [
        'реклама', 'спонсор', 'купон', 'скидк', 'акция', 'распродаж',
        'заказать', 'купить', 'продаж', 'магазин', 'доставк',
        'ad', 'sponsor', 'sale', 'discount', 'promo', 'buy', 'shop',
        'партнер', 'промокод', 'офер', 'доставка', 'цена', 'руб'
    ]
    
    for post in posts:
        if not post.text:
            continue
        
        text = post.text.lower()
        
        # Пропускаем слишком короткие посты
        if len(text) < MIN_POST_LENGTH:
            continue
        
        # Проверяем на рекламные ключевые слова
        is_ad = any(keyword in text for keyword in ad_keywords)
        
        # Проверяем на обилие ссылок
        link_count = text.count('http') + text.count('t.me') + text.count('@')
        
        # Проверяем на обилие специальных символов (возможно, спам)
        special_chars = sum(1 for char in text if char in '!$%&*#')
        
        if not is_ad and link_count < MAX_LINKS_PER_POST and special_chars < MAX_SPECIAL_CHARS:
            filtered.append(post)
    
    return filtered

In [13]:
async def collect_posts_for_channels(channels, days_back=DAYS_BACK, posts_per_channel=POSTS_PER_CHANNEL):
    """
    Собирает посты для всех каналов.
    """
    print(f"\nСобираю посты для {len(channels)} каналов...")
    
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_back)
    
    channels_with_posts = []
    
    for i, channel_info in enumerate(channels, 1):
        print('channel_info:', channel_info)
        channel_id = channel_info['handle'][1:]
        category = channel_info['category']
        
        print(f"  [{i}/{len(channels)}] {category}: {channel_id}")
        
        try:
            posts = await get_channel_posts(channel_id, start_date, end_date, limit=POST_LIMIT_PER_REQUEST)
            filtered_posts = filter_non_ad_posts(posts)
            
            if filtered_posts:
                # Берем не более posts_per_channel постов
                selected_posts = filtered_posts[:posts_per_channel]
                channel_info['posts'] = selected_posts
                channels_with_posts.append(channel_info)
                print(f"    ✓ {len(selected_posts)}/{len(posts)} постов")
            else:
                print(f"    ✗ Нет подходящих постов")
                channel_info['posts'] = []
                
        except Exception as e:
            print(f"    ✗ Ошибка: {e}")
            channel_info['posts'] = []
        
        # Небольшая задержка между запросами
        await asyncio.sleep(1)
    
    print(f"\n✓ Каналов с постами: {len(channels_with_posts)}")
    
    # Считаем общее количество постов
    total_posts = sum(len(c['posts']) for c in channels_with_posts)
    print(f"✓ Всего нерекламных постов: {total_posts}")
    
    return channels_with_posts

# Запускаем сбор постов (асинхронно)
print("Запускаю сбор постов из каналов...")
channels_with_posts = await collect_posts_for_channels(all_channels, days_back=DAYS_BACK, posts_per_channel=POSTS_PER_CHANNEL)

Запускаю сбор постов из каналов...

Собираю посты для 43 каналов...
channel_info: {'channel_id': '2', 'category': 'apps channels', 'source_file': 'RU_apps_channels.csv', 'handle': '@TikTokModCloud', 'posts': []}
  [1/43] apps channels: TikTokModCloud
Message(id=3054, peer_id=PeerChannel(channel_id=1160853809), date=datetime.datetime(2025, 12, 7, 18, 15, 34, tzinfo=datetime.timezone.utc), message='TikTokMod 42.9.3 (Asia)\n- Disabled all ads\n- Bypassed regional restrictions\n- Bypassed all download restrictions\n- Downloads of videos and photos are now saved without watermark to Movies/TikTok instead of DCIM/Camera; photos are saved to Pictures/TikTok\n- Added the ability to download photos from comments without a watermark; photos are saved to Pictures/TikTok\n- Added author tag to the name of downloaded files\n- Removed restrictions on screenshots and screen recording (e.g., in livestreams)\n- Disabled all types of analytics\n- Disabled InAppBillingService\n- Disabled mandatory login\

In [20]:
def create_golden_dataset(products, channels_with_posts, category_mapping, target_pairs=TARGET_PAIRS):
    """
    Создает golden dataset с использованием маппинга категорий.
    """
    print(f"\nСоздаю golden dataset (цель: {target_pairs} пар)...")
    
    # Подготовка данных
    # Группируем каналы по категориям
    channels_by_category = {}
    for channel in channels_with_posts:
        category = channel['category']
        if category not in channels_by_category:
            channels_by_category[category] = []
        channels_by_category[category].append(channel)
    
    # Группируем товары по категориям
    products_by_category = {}
    for product in products:
        category = product.get('subj_root_name', 'Без категории')
        if category not in products_by_category:
            products_by_category[category] = []
        products_by_category[category].append(product)
    
    # Создаем пары
    golden_pairs = []
    attempts = 0
    max_attempts = target_pairs * MAX_ATTEMPTS_MULTIPLIER  # Ограничиваем количество попыток
    
    while len(golden_pairs) < TARGET_PAIRS and attempts < MAX_ATTEMPTS_MULTIPLIER :
        attempts += 1
        
        # Решаем, будет ли пара подходящей (50/50)
        is_matching = random.random() < MATCHING_RATIO
        
        if is_matching:
            # ПОДХОДЯЩАЯ ПАРА: используем маппинг категорий
            
            # Выбираем случайную категорию товара, которая есть в маппинге
            available_product_categories = [cat for cat in products_by_category.keys() 
                                          if cat in category_mapping and products_by_category[cat]]
            
            if not available_product_categories:
                continue
                
            product_category = random.choice(available_product_categories)
            
            # Выбираем случайный товар из этой категории
            product = random.choice(products_by_category[product_category])
            
            # Получаем подходящие категории каналов из маппинга
            suitable_channel_categories = category_mapping.get(product_category, [])
            
            if not suitable_channel_categories:
                continue
            
            # Выбираем случайную подходящую категорию канала
            channel_category = random.choice(suitable_channel_categories)
            
            # Ищем каналы в этой категории
            if channel_category in channels_by_category and channels_by_category[channel_category]:
                channel = random.choice(channels_by_category[channel_category])
                
                # Выбираем случайный пост из канала
                if channel.get('posts'):
                    post = random.choice(channel['posts'])
                    
                    golden_pairs.append({
                        'query': product.get('description', product.get('imt_name', '')),
                        'post': post.text if post.text else '',
                        'score': 1.0,  # Подходящая пара
                        'is_matching': True,
                        'product_category': product_category,
                        'channel_category': channel_category,
                        'product_id': product.get('imt_id', ''),
                        'channel_id': channel.get('channel_id', ''),
                        'post_date': post.date.isoformat() if post.date else ''
                    })
        
        else:
            # НЕПОДХОДЯЩАЯ ПАРА: случайные категории
            
            # Выбираем случайный товар
            all_product_cats = list(products_by_category.keys())
            product_category = random.choice(all_product_cats)
            product = random.choice(products_by_category[product_category])
            
            # Получаем подходящие категории из маппинга
            suitable_categories = category_mapping.get(product_category, [])
            
            # Выбираем случайную категорию канала, которая НЕ входит в подходящие
            all_channel_cats = list(channels_by_category.keys())
            
            # Фильтруем неподходящие категории
            unsuitable_categories = [cat for cat in all_channel_cats if cat not in suitable_categories]
            
            if not unsuitable_categories:
                # Если все категории подходят, берем любую, но отмечаем как неподходящую
                channel_category = random.choice(all_channel_cats) if all_channel_cats else None
            else:
                channel_category = random.choice(unsuitable_categories)
            
            if channel_category and channel_category in channels_by_category:
                channel = random.choice(channels_by_category[channel_category])
                
                # Выбираем случайный пост из канала
                if channel.get('posts'):
                    post = random.choice(channel['posts'])
                    
                    golden_pairs.append({
                        'query': product.get('description', product.get('imt_name', '')),
                        'post': post.text if post.text else '',
                        'score': 0.0,  # Неподходящая пара
                        'is_matching': False,
                        'product_category': product_category,
                        'channel_category': channel_category,
                        'product_id': product.get('imt_id', ''),
                        'channel_id': channel.get('channel_id', ''),
                        'post_date': post.date.isoformat() if post.date else ''
                    })
    
    print(f"\nСоздано пар: {len(golden_pairs)}")
    print('Пары:', golden_pairs)
    
    # Статистика
    matching_count = sum(1 for p in golden_pairs if p['is_matching'])
    non_matching_count = len(golden_pairs) - matching_count
    
    print(f"✓ Подходящих пар (score=1.0): {matching_count} ({matching_count/len(golden_pairs)*100:.1f}%)")
    print(f"✓ Неподходящих пар (score=0.0): {non_matching_count} ({non_matching_count/len(golden_pairs)*100:.1f}%)")
    
    return golden_pairs

# Создаем golden dataset
golden_data = create_golden_dataset(products, channels_with_posts, category_mapping, target_pairs=2000)
print('golden_data:', golden_data)


Создаю golden dataset (цель: 2000 пар)...

Создано пар: 2
Пары: [{'query': 'Помпа механическая для 19-ти литровых бутылей. Одна из самых надежных помп на рынке. Не требует электропитания. Выполнена из пищевого пластика.', 'post': '**СЛУГА НАРОДА. АГОНИЯ. 2 СЕЗОН. 3 СЕРИЯ**\n\nДональд Трамп – младший:\n\nЭтим летом я был со своей девушкой в Монако. Мы катались по окрестностям, и в среднем за день 50% суперкаров, таких как "Бугатти", "Феррари" и т.д., имели украинские номера. Как вы считаете, это действительно было заработано на Украине? Я был на Украине 20 лет назад. Не то чтобы в этой стране изобилие богатств.\nСын предыдущего президента США, Хантер Байден, был на Украине существенно чаще, и он своё изобилие там нашёл. Основанное на западной помощи. Настолько обильной, что Байдену потребовалась амнистия от собственного отца.\n\nА Трамп – младший продолжает говорить правду:\n\nБогатые сбежали. Они оставили воевать тех, кого считали крестьянами, и не видели никакой причины останавливать

In [23]:
# def save_golden_dataset(golden_data, output_folder, filename="golden_dataset.csv"):
#     """
#     Сохраняет golden dataset в CSV файл.
#     """
#     if not golden_data:
#         print("✗ Нет данных для сохранения")
#         return None
    
#     # Создаем DataFrame
#     df = pd.DataFrame(golden_data)
    
#     # Убедимся, что у нас есть нужные колонки для обучения
#     required_columns = ['query', 'post', 'score']
#     for col in required_columns:
#         if col not in df.columns:
#             print(f"✗ Отсутствует обязательная колонка: {col}")
    
#     # Сохраняем в CSV
#     output_path = os.path.join(output_folder, filename)
#     df.to_csv(output_path, index=False, encoding='utf-8-sig')
    
#     print(f"\n✓ Golden dataset сохранен: {output_path}")
#     print(f"✓ Размер: {len(df)} строк, {len(df.columns)} столбцов")
    
#     # Дополнительная информация
#     print(f"\nСтруктура файла:")
#     print(f"  Обязательные колонки для обучения:")
#     print(f"    - query: текст запроса (описание товара)")
#     print(f"    - post: текст поста из канала")
#     print(f"    - score: 1.0 (подходит) или 0.0 (не подходит)")
    
#     print(f"\n  Дополнительные колонки для анализа:")
#     for col in df.columns:
#         if col not in ['query', 'post', 'score']:
#             print(f"    - {col}")
    
#     return output_path

def save_golden_dataset(golden_data, output_folder, filename="gold_pairs.csv"):
    """
    Сохраняет golden dataset в CSV файл.
    """
    if not golden_data:
        print("✗ Нет данных для сохранения")
        return None
    
    # Создаем DataFrame
    df = pd.DataFrame(golden_data)
    
    # Убедимся, что у нас есть нужные колонки для обучения
    required_columns = ['query', 'post', 'score']
    for col in required_columns:
        if col not in df.columns:
            print(f"✗ Отсутствует обязательная колонка: {col}")
    
    # Сохраняем в CSV с защитой от проблемных символов
    output_path = os.path.join(output_folder, filename)
    
    # Вариант 1: С разделителем табуляции (менее проблемный)
    df.to_csv(output_path, index=False, encoding='utf-8-sig', sep='\t')
    
    # ИЛИ Вариант 2: С кавычками вокруг всех текстовых полей
    # df.to_csv(output_path, index=False, encoding='utf-8-sig', 
    #           quoting=csv.QUOTE_ALL, quotechar='"')
    
    print(f"\n✓ Golden dataset сохранен: {output_path}")
    print(f"✓ Размер: {len(df)} строк, {len(df.columns)} столбцов")
    print(f"✓ Разделитель: табуляция (TSV формат)")
    
    return output_path

# Сохраняем dataset
golden_file = save_golden_dataset(golden_data, golden_folder, "gold_pairs.csv")


✓ Golden dataset сохранен: d:\Projects\thesis\data\golden\gold_pairs.csv
✓ Размер: 2 строк, 9 столбцов
✓ Разделитель: табуляция (TSV формат)


In [ ]:
def save_golden_dataset(golden_data, output_folder):
    """
    Сохраняет golden dataset в нескольких форматах.
    """
    if not golden_data:
        print("✗ Нет данных для сохранения")
        return None
    
    df = pd.DataFrame(golden_data)
    
    # 1. JSON (для проверки человеком)
    json_path = os.path.join(output_folder, "gold_pairs.json")
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(golden_data, f, ensure_ascii=False, indent=2)
    
    
    
    print(f"\n✓ JSON сохранен: {json_path}")
    print(f"✓ Размер: {len(golden_data)} пар")
    
    # Показываем первые 2 строки для проверки
    print("\nПервые 2 строки (из JSON):")
    for i, item in enumerate(golden_data[:2], 1):
        print(f"\nПара {i}:")
        print(f"  Query: {item['query'][:80]}...")
        print(f"  Post: {item['post'][:80]}...")
        print(f"  Score: {item['score']}")
    
    return json_path

# Использование
json_file = save_golden_dataset(golden_data, golden_folder)


✓ JSON сохранен: d:\Projects\thesis\data\golden\gold_pairs.json
✓ Размер: 2 пар

Первые 2 строки (из JSON):

Пара 1:
  Query: Помпа механическая для 19-ти литровых бутылей. Одна из самых надежных помп на ры...
  Post: **СЛУГА НАРОДА. АГОНИЯ. 2 СЕЗОН. 3 СЕРИЯ**

Дональд Трамп – младший:

Этим летом...
  Score: 0.0

Пара 2:
  Query: Весы кухонные электорнные 5 кг. Для тех, кто придерживается строгих диет, и для ...
  Post: SpaceX опубликовала список самых популярных поисковых запросов украинских военно...
  Score: 0.0


NameError: name 'csv_path' is not defined

In [28]:
def save_all_posts_json(channels_with_posts, output_folder, filename="all_posts.json"):
    """
    Сохраняет все посты в JSON файл.
    """
    all_posts_data = []
    
    for channel in channels_with_posts:
        for post in channel.get('posts', []):
            if post.text:  # Сохраняем только посты с текстом
                all_posts_data.append({
                    'post_text': post.text,
                    'channel_id': channel.get('channel_id', ''),
                    'channel_handle': channel.get('handle', ''),  # <-- ДОБАВЛЕНО
                    'channel_category': channel.get('category', ''),
                    'post_date': post.date.isoformat() if post.date else ''
                })
    
    if all_posts_data:
        output_path = os.path.join(output_folder, filename)
        
        # Сохраняем в JSON
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(all_posts_data, f, ensure_ascii=False, indent=2)
        
        print(f"\n✓ Все посты сохранены в JSON: {output_path}")
        print(f"✓ Количество постов: {len(all_posts_data)}")
        
        return output_path
    else:
        print("\n✗ Нет постов для сохранения")
        return None

# Сохраняем все посты
posts_file = save_all_posts_json(channels_with_posts, golden_folder, "all_posts.json")


✓ Все посты сохранены в JSON: d:\Projects\thesis\data\golden\all_posts.json
✓ Количество постов: 4


In [17]:
print("\n" + "=" * 60)
print("СТАТИСТИКА И ПРОВЕРКА КАЧЕСТВА")
print("=" * 60)

# Загружаем сохраненный golden dataset для проверки
if golden_file and os.path.exists(golden_file):
    df_check = pd.read_csv(golden_file, encoding='utf-8-sig')
    
    print(f"\nПроверка golden dataset:")
    print(f"  Всего пар: {len(df_check)}")
    print(f"  Подходящих (score=1.0): {len(df_check[df_check['score'] == 1.0])}")
    print(f"  Неподходящих (score=0.0): {len(df_check[df_check['score'] == 0.0])}")
    
    # Проверяем длину текстов
    df_check['query_length'] = df_check['query'].astype(str).apply(len)
    df_check['post_length'] = df_check['post'].astype(str).apply(len)
    
    print(f"\nСтатистика по длине текстов:")
    print(f"  Запросы (query):")
    print(f"    - Средняя длина: {df_check['query_length'].mean():.1f} символов")
    print(f"    - Минимальная: {df_check['query_length'].min()} символов")
    print(f"    - Максимальная: {df_check['query_length'].max()} символов")
    
    print(f"  Посты (post):")
    print(f"    - Средняя длина: {df_check['post_length'].mean():.1f} символов")
    print(f"    - Минимальная: {df_check['post_length'].min()} символов")
    print(f"    - Максимальная: {df_check['post_length'].max()} символов")
    
    # Примеры пар
    print(f"\nПримеры пар из golden dataset:")
    print("-" * 80)
    
    # Подходящая пара
    matching_example = df_check[df_check['score'] == 1.0].iloc[0] if len(df_check[df_check['score'] == 1.0]) > 0 else None
    if matching_example is not None:
        print(f"ПОДХОДЯЩАЯ ПАРА (score=1.0):")
        print(f"  Категория товара: {matching_example['product_category']}")
        print(f"  Категория канала: {matching_example['channel_category']}")
        print(f"  Запрос: {matching_example['query'][:100]}...")
        print(f"  Пост: {matching_example['post'][:100]}...")
        print()
    
    # Неподходящая пара
    non_matching_example = df_check[df_check['score'] == 0.0].iloc[0] if len(df_check[df_check['score'] == 0.0]) > 0 else None
    if non_matching_example is not None:
        print(f"НЕПОДХОДЯЩАЯ ПАРА (score=0.0):")
        print(f"  Категория товара: {non_matching_example['product_category']}")
        print(f"  Категория канала: {non_matching_example['channel_category']}")
        print(f"  Запрос: {non_matching_example['query'][:100]}...")
        print(f"  Пост: {non_matching_example['post'][:100]}...")
    
    # Распределение по категориям
    print(f"\nРаспределение пар по категориям товаров (топ-10):")
    category_counts = df_check['product_category'].value_counts().head(10)
    for category, count in category_counts.items():
        percentage = count / len(df_check) * 100
        print(f"  {category}: {count} пар ({percentage:.1f}%)")
    
    print(f"\n✓ Golden dataset готов для ручной разметки!")
    print(f"✓ Файлы сохранены в папке: {golden_folder}")
    
else:
    print("✗ Файл golden dataset не найден")


СТАТИСТИКА И ПРОВЕРКА КАЧЕСТВА

Проверка golden dataset:
  Всего пар: 2
  Подходящих (score=1.0): 0
  Неподходящих (score=0.0): 2

Статистика по длине текстов:
  Запросы (query):
    - Средняя длина: 179.5 символов
    - Минимальная: 168 символов
    - Максимальная: 191 символов
  Посты (post):
    - Средняя длина: 278.5 символов
    - Минимальная: 186 символов
    - Максимальная: 371 символов

Примеры пар из golden dataset:
--------------------------------------------------------------------------------
НЕПОДХОДЯЩАЯ ПАРА (score=0.0):
  Категория товара: Посуда и инвентарь
  Категория канала: blogs channels
  Запрос: Поможет в решении большинства повседневных задач на кухне. Рекомендуем для семьи, и в подарок любимы...
  Пост: Мы монтируем так… что к концу месяца я буду выглядеть так!😂(Это мое лицо после 7 дней в бункере) 
Кр...

Распределение пар по категориям товаров (топ-10):
  Посуда и инвентарь: 1 пар (50.0%)
  Красота: 1 пар (50.0%)

✓ Golden dataset готов для ручной разметки!
✓